# Job Classifier

In [497]:
import pandas as pd
import numpy as np
from nltk.tag import StanfordPOSTagger

In [498]:
import nltk
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')

## 1.1 Prepare sample data from POSTags, NO longer needed

In [499]:
inputfile = "./tags/nnp-summary.txt"

#df_nn_summary = pd.read_table(nn_summary, sep="|",index_col=False, header=None, error_bad_lines=False)

#names = ["token"]*1314
#df_nn_summary = pd.read_table(nn_summary, sep="|",index_col=False, header=None, names = names) #error_bad_lines=False)

In [500]:
import csv
with open(inputfile) as f:
    reader = csv.reader(f, delimiter="|")
    d = list(reader)
print d[0][2] # test

ETL


In [501]:
# the first line of our data
print '|'.join(d[0])

SQL|SQL Developer|ETL|SSIS|SQL|Server|SQL|Server|Web|JavaScript|JQuery|HTML5|CSS3|JavaScript|•|Extensive|Microsoft|Business|Intelligence|SQL|Server|Integration|Services|SSIS|SQL|Server|Reporting|Services|SSRS|ETL|SSIS|Custom|Report|Tabular|Reports|Matrix|Reports|Ad|SQL|Server|Reporting|Services|SSRS|DATA|•|Expert|Bulk|BCP|Data|Transformation|Services|DTS|SSIS|•|Expertise|SQL|Server|Data|Modeling|Star|Schema/Snowflake|Schema|Fact|Dimensions|Visio|Software|Development|Life|Cycle|SDLC|Agile|Methodology|Waterfall|ETL|SSIS|•|Extensive|SSIS|File|Transfer|Protocol|FTP|Secure|File|Transfer|Protocol|SFTP


In [502]:
# TECH category list
d_target = [item[0] for item in d]
print d_target[0:10]
print len(d_target)

['SQL', 'SQL', 'SQL', 'SQL', 'SQL', 'SQL', 'SQL', 'SQL', 'SQL', 'SQL']
1271


In [503]:
# TITLE category list
d_title = [' '.join(item[1:2]) for item in d]
print d_title[0:10]
print len(d_title)

['SQL Developer', 'SQL Developer', 'SQL / BI Developer', 'Sr. SQL Developer', 'Sr. SQL Server DBA', 'Sr. SQL Developer', 'PL/SQL Developer', 'Sr. SQL Server DBA', 'SQL Server DBA', 'Sql Server/bi Developer']
1271


In [504]:
# token list
d_train = [item[2:] for item in d]
print d_train[0][:10]
print len(d_train)

['ETL', 'SSIS', 'SQL', 'Server', 'SQL', 'Server', 'Web', 'JavaScript', 'JQuery', 'HTML5']
1271


## 1.2 [NEW] Read origin text corpus from MongoDB instead of using POSTags
files are in JSON format

In [505]:
import pymongo
client = pymongo.MongoClient('192.168.1.232',27017)

In [506]:
db = client.ir

In [507]:
db.collection_names()

[u'movie',
 u'TaggedResumes',
 u'OperatingSystems',
 u'Jobs',
 u'Mails',
 u'KeyWords',
 u'IndeedJobs',
 u'TaggedMails',
 u'ResumeLinks',
 u'Technologies']

In [508]:
resumes = db.TaggedResumes

In [509]:
summary = resumes.find({},{"searchKey": 1, "title": 1, "details.summary": 1})

summary = np.array(list(summary))

summary[0].values()#[2].values()

for i in range(len(summary)):
#summary[0]
    summary[i] = summary[i].values()
    summary[i][2] = summary[i][2].values()

In [510]:
print len(summary)
#summary

1071


In [511]:
f_train = ["".join(item[2]) for item in summary]
print len(f_train), f_train[0]

1071 • Over 6 years of Experience in requirements gathering, design, development, implementation and testing the web applications and ETL tools which includes SSIS, SQL Server 2012, and SQL Server 2008. 
• Experience in documenting the business requirements for the development team. 
• Experience in Web design, development, testing/debugging, implementation, and operations in medium-to-large enterprise environments 
• Experienced in JavaScript, JQuery, HTML5, CSS3, and JavaScript. 
• Extensive experience in Microsoft Business Intelligence technologies like SQL Server Integration Services (SSIS) and SQL Server Reporting Services (SSRS). 
• Worked on all activities related to the development, implementation, administration and support of ETL processes for large-scale Data Warehouses using SSIS 2012. 
• Experience in developing Custom Report and different types of Tabular Reports, Matrix Reports, Ad hoc reports and distributed reports in multiple formats using SQL Server Reporting Service

In [512]:
f_tech = [item[0] for item in summary]
print len(f_tech), f_tech[0:10]

1071 [u'SQL', u'SQL', u'SQL', u'SQL', u'SQL', u'SQL', u'SQL', u'SQL', u'SQL', u'SQL']


In [513]:
f_title = [item[3] for item in summary]
print len(f_title), f_title[0:10]

1071 [u'SQL Developer', u'SQL Developer', u'SQL / BI Developer', u'Sr. SQL Developer', u'Sr. SQL Server DBA', u'Sr. SQL Developer', u'PL/SQL Developer', u'Sr. SQL Server DBA', u'SQL Server DBA', u'Sql Server/bi Developer']


In [514]:
set(f_tech)

{u'.NET',
 u'Agile',
 u'Automation',
 u'C#',
 u'C++',
 u'CRM',
 u'CSS',
 u'Cloud',
 u'HTML',
 u'Java',
 u'JavaScript',
 u'Linux',
 u'Microsoft SQL Server',
 u'Microsoft Visio',
 u'Microsoft Windows',
 u'Oracle',
 u'SDLC',
 u'SQL',
 u'Unix',
 u'Web services',
 u'Workflow',
 u'XML'}

## 2. Processing input data : job requirement.docx/doc

### 2.1 Read in raw text into single string

In [515]:
#with open('./Python_job_req1.txt', 'r') as myfile: # python
#with open('./java-developer-req1.txt', 'r') as myfile: # java
#with open('./sql-req1.txt', 'r') as myfile: # SQL
with open('./oracle-req1.txt', 'r') as myfile: # Oracle
    test_raw=myfile.read().replace('\n', ' ')
    
test_raw = test_raw.replace("\x95\t"," ")
test_raw

"Senior Oracle Database Developer  Softweb Solutions Inc - Parsippany, NJ - Must have strong Oracle PL/SQL application development skills and excellent knowledge of Stored Procedures, Functions, Packages, Triggers, Views,Materialized Views, Cursors, and XML features and writing and tuning complex SQL queries. - Additionally, experience with autonomous transactions and error handling is highly desirable. - Experience in Implementing data models and database designs - Experienced with Stored procedure creation/debugging/ optimization/tuning - Experience with UNIX shell and CVS is helpful - Experience with Oracle 12c is desirable - Writing and creating data load scripts and stored procedures; ensuring that data is loaded in a timely manner daily - Working with application developers to assist in database design, query tuning, index assignment, and trigger and procedure creation - Ability to research and trouble-shoot application problems using PL/SQL programming skills - Ensuring that the

### 2.2 POSTagging and keep nouns [no longer needed]

In [516]:
tokens = nltk.word_tokenize(test_raw.decode('UTF-8'))
print ' '.join(tokens[:20])

Senior Oracle Database Developer Softweb Solutions Inc - Parsippany , NJ - Must have strong Oracle PL/SQL application development skills


In [517]:
input_postag = nltk.pos_tag(tokens)
input_postag[:11]

[(u'Senior', 'JJ'),
 (u'Oracle', 'NNP'),
 (u'Database', 'NNP'),
 (u'Developer', 'NNP'),
 (u'Softweb', 'NNP'),
 (u'Solutions', 'NNP'),
 (u'Inc', 'NNP'),
 (u'-', ':'),
 (u'Parsippany', 'NN'),
 (u',', ','),
 (u'NJ', 'NNP')]

In [518]:
# filtering list, keep NNP only
input_postag_nn = [x[0] for x in input_postag if x[1] == 'NNP']
#input_postag_nn = list(set(input_postag_nn))
print '|'.join(input_postag_nn)

Oracle|Database|Developer|Softweb|Solutions|Inc|NJ|Oracle|PL/SQL|Stored|Procedures|Functions|Packages|Triggers|Views|Materialized|Views|Cursors|XML|SQL|Implementing|Stored|UNIX|CVS|Oracle|PL/SQL|Data|Integrity|Required|Skills|Software|Development|Oracle|RDBMS|SQL|OLTP|Agile|Methodology|Job|Type|Job|Location|NJ|Required|Bachelor|PL/SQL


## 3. Classification using Naive Bayes in scikit-learn 
[Working With Text](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#working-with-text-data)  
[Multinomial Naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes)  
[MultinomialNB Example](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)

In [519]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [520]:
d_train_l = [" ".join(item) for item in d_train]

In [521]:
#d_train_l

In [522]:
input_nn = " ".join(input_postag_nn)

In [523]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(ngram_range=(1, 2))#(stop_words = 'english')
vectorizer_new = CountVectorizer(stop_words='english',ngram_range=(1, 2))#(stop_words = 'english')

In [524]:
#X_vectorized = vectorizer.fit_transform(X_nn_summary_list)
d_train_l_vector = vectorizer.fit_transform(d_train_l)

# vectorize full text training data
f_train_vector = vectorizer_new.fit_transform(f_train)

In [525]:
print d_train_l_vector.shape
print len(d_target)
print f_train_vector.shape
print len(f_tech)

(1271, 41473)
1271
(1071, 55508)
1071


In [526]:
#print vectorizer.get_feature_names()

In [527]:
vectorizer_new.vocabulary_.get(u'java')

25093

In [528]:
from sklearn.feature_extraction.text import TfidfTransformer
#tf_transformer = TfidfTransformer(use_idf=False).fit(d_train_l_vector)
#d_train_tf = tf_transformer.transform(d_train_l_vector)
#d_train_tf.shape

In [529]:
tfidf_transformer = TfidfTransformer()
tfidf_transformer_new = TfidfTransformer()
d_train_tfidf = tfidf_transformer.fit_transform(d_train_l_vector)
print d_train_tfidf.shape
print len(d_target),type(d_target)

f_train_tfidf = tfidf_transformer_new.fit_transform(f_train_vector)
print f_train_tfidf.shape
print len(f_tech),type(f_tech)
print len(f_title)

(1271, 41473)
1271 <type 'list'>
(1071, 55508)
1071 <type 'list'>
1071


In [530]:
f_train_tfidf.toarray()[0].sum()

15.633994172019005

In [531]:
# input_nn = " ".join(tokens)  # comment out this line if you want to use nnp instead of all test corpus
# vectorize test data with nnp only
input_nn_list = [input_nn]
input_nn_vectorized = vectorizer.transform(input_nn_list)

In [532]:
# test_raw to vector
test_raw_list = [test_raw]
test_raw_vectorized = vectorizer_new.transform(test_raw_list)

In [533]:
# test with data from training set
testString = d_train_l[0].replace("\xe2\x80\xa2","")
testStringList = [testString]
testVector = vectorizer.transform(testStringList)

testString_new = f_train[0]
testStringList_new = [testString_new]
testVector_new = vectorizer_new.transform(testStringList_new)

testVector2 = vectorizer.transform(["This is a job requirement for python programmer\
    who needs to know flask, numpy and pandas."])
testVector2_new = vectorizer_new.transform(["This is a job requirement for python programmer\
    who needs to know python and python libraries such as flask, numpy and pandas."])

In [534]:
# fit multinomial Naive Bayes model
clf1 = MultinomialNB()
clf2 = MultinomialNB()
clf3 = MultinomialNB()

model1 = clf1.fit(d_train_l_vector, d_target) # TECH model
# validate model using training data
print model1.predict(testVector),model1.predict(testVector2) # Python # SQL

model2 = clf2.fit(d_train_tfidf, d_target) # TECH model w/ df-idf
print model2.predict(testVector),model2.predict(testVector2) # Python # SQL

model3 = clf3.fit(d_train_tfidf, d_title) # TITILE model w/ df-idf
print model3.predict(testVector),model3.predict(testVector2) # Python # SQL

['SQL'] ['Python']
['SQL'] ['Python']
['Net Developer'] ['Python Developer']


In [535]:
# predict test data
print "Model1-Tech1:", model1.predict(input_nn_vectorized)
print "Model2-Tech2:", model2.predict(input_nn_vectorized)
print "Model3-Title:", model3.predict(input_nn_vectorized)

Model1-Tech1: ['PL/SQL']
Model2-Tech2: ['PL/SQL']
Model3-Title: ['Oracle PL/SQL Developer']


In [536]:
# test data, 2grams is not shown here. 2grams are generated when calling vectorizer
print " ".join(tokens[:2]), "|","|".join(input_nn.split(" "))

Senior Oracle | Oracle|Database|Developer|Softweb|Solutions|Inc|NJ|Oracle|PL/SQL|Stored|Procedures|Functions|Packages|Triggers|Views|Materialized|Views|Cursors|XML|SQL|Implementing|Stored|UNIX|CVS|Oracle|PL/SQL|Data|Integrity|Required|Skills|Software|Development|Oracle|RDBMS|SQL|OLTP|Agile|Methodology|Job|Type|Job|Location|NJ|Required|Bachelor|PL/SQL


#### using full text

In [537]:
# fit multinomial Naive Bayes model
clf1_new = MultinomialNB()
clf2_new = MultinomialNB()
clf3_new = MultinomialNB()

model1_new = clf1_new.fit(f_train_vector, f_tech) # TECH model


In [538]:
# validate model using training data
print model1_new.predict(testVector_new),model1_new.predict(testVector2_new) # Python # SQL

model2_new = clf2_new.fit(f_train_tfidf, f_tech) # TECH model w/ df-idf
print model2_new.predict(testVector_new),model2_new.predict(testVector2_new) # Python # SQL

model3_new = clf3_new.fit(f_train_tfidf, f_title) # TITILE model w/ df-idf
print model3_new.predict(testVector_new),model3_new.predict(testVector2_new) # Python # SQL

[u'SQL'] [u'Linux']
[u'SQL'] [u'Linux']
[u'Net Developer'] [u'Linux Administrator']


In [545]:
# predict test data
print "Model1_new-Tech1:", model1_new.predict(test_raw_vectorized)
print "Model2_new-Tech2:", model2_new.predict(test_raw_vectorized)
print "Model3_new-Title:", model3_new.predict(test_raw_vectorized)

Model1_new-Tech1: [u'SQL']
Model2_new-Tech2: [u'SQL']
Model3_new-Title: [u'Net Developer']


## Classifiying using Random Forest

In [540]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier
forest1 = RandomForestClassifier(n_estimators = 100) #random_state=15326
forest2 = RandomForestClassifier(n_estimators = 100)
forest3 = RandomForestClassifier(n_estimators = 100)

Training the random forest...


In [541]:
forest1 = forest1.fit(f_train_vector, f_tech)
forest2 = forest2.fit(f_train_tfidf, f_tech)
forest3 = forest3.fit(f_train_tfidf, f_title)

In [542]:
# validate model with data from training set
print forest1.predict(testVector_new), forest1.predict(testVector2_new)  #SQL Python

[u'SQL'] [u'CSS']


In [543]:
# test data
print "Forest1-Tech1:",forest1.predict(test_raw_vectorized)
print "Forest2-Tech2:",forest2.predict(test_raw_vectorized)
print "Forest3-Title:",forest3.predict(test_raw_vectorized)

Forest1-Tech1: [u'SQL']
Forest2-Tech2: [u'Oracle']
Forest3-Title: [u'Oracle DBA']


In [544]:
test_raw

"Senior Oracle Database Developer  Softweb Solutions Inc - Parsippany, NJ - Must have strong Oracle PL/SQL application development skills and excellent knowledge of Stored Procedures, Functions, Packages, Triggers, Views,Materialized Views, Cursors, and XML features and writing and tuning complex SQL queries. - Additionally, experience with autonomous transactions and error handling is highly desirable. - Experience in Implementing data models and database designs - Experienced with Stored procedure creation/debugging/ optimization/tuning - Experience with UNIX shell and CVS is helpful - Experience with Oracle 12c is desirable - Writing and creating data load scripts and stored procedures; ensuring that data is loaded in a timely manner daily - Working with application developers to assist in database design, query tuning, index assignment, and trigger and procedure creation - Ability to research and trouble-shoot application problems using PL/SQL programming skills - Ensuring that the